<a href="https://colab.research.google.com/github/mugalan/mugalan/blob/main/LMS_Monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up

## Initialization


In [ ]:
import numpy as np
import pandas as pd
import csv
import plotly.graph_objects as go
import datetime

### Mounting the Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Functions used

In [ ]:
def grouped_itemized_graphs(dataPDF, gpFieldName, gpFldList, plotFieldName, plotFldList, yLabel, figTitle):
  xx1=[]; yy1=[]; cc1=[]; traceNames1=[];
  aaPDF=pd.DataFrame(yy1) #,index=plotFldList, columns=gpFldList)
  
  for xx in gpFldList:
    if gpFieldName==None:
      temp=dataPDF
    else:
      temp=dataPDF[dataPDF[gpFieldName]==xx];    
    xxx1=[]; yyy1=[]; ccc1=[];
    
    for tstdt in plotFldList:
      xxx1+=[tstdt]
      ccc1+=[temp[temp[plotFieldName]==tstdt]];
      yyy1+=[len(temp[temp[plotFieldName]==tstdt])]
    xx1+=[xxx1];
    yy1+=[yyy1];
    cc1+=[ccc1]
    traceNames1+=[xx]
    
  fig = go.Figure()
  for i,x in enumerate(xx1):
    fig.add_trace(go.Bar(y=yy1[i], x=x, name=traceNames1[i]))
  
  
  fig.update_layout(title=figTitle, xaxis_title='', yaxis_title=yLabel, barmode='stack')
  fig.show()

  tmpx0=pd.DataFrame(yy1)
  tmpx00=tmpx0.transpose() #,index=plotFldList, columns=gpFldList)
  aaPDF=pd.DataFrame(tmpx00.to_numpy(),index=plotFldList, columns=gpFldList)
  return fig, aaPDF

# Import Data

In [ ]:
#Import User information
usersPDF0=pd.read_csv('/content/drive/My Drive/SLTC/LMS Usage/Users.csv'); usersPDF0['User full name']=usersPDF0['firstname']+' '+usersPDF0['lastname']
#usersPDF0=pd.read_csv('/content/drive/My Drive/OriseUsers.csv'); usersPDF0['User full name']=usersPDF0['firstname']+' '+usersPDF0['lastname']
#usersPDF0=pd.read_csv('/content/drive/My Drive/ME320Users.csv'); usersPDF0['User full name']=usersPDF0['ENo[Student]/Title[Staff]']+' '+usersPDF0['Name']
usersPDF=usersPDF0.fillna('')

listdeRegisteredStudents=usersPDF[usersPDF['type']=='de-registered']['User full name'].to_list()
listStudents=usersPDF[(usersPDF['type']=='student') & (usersPDF['profile_field_Active']=='A')]['User full name'].to_list()
listFaculty=usersPDF[usersPDF['type']=='faculty']['User full name'].to_list()
listdeRegisteredStudents.sort()
listStudents.sort()
listFaculty.sort()

In [ ]:
#Open last file
dataPDF=pd.read_csv('/content/drive/My Drive/SLTC/LMS Usage/SLTC-LMS-logs-concatenated.csv')
#dataPDF=pd.read_csv('/content/drive/My Drive/logs_ORISE.csv')
#dataPDF=pd.read_csv('/content/drive/My Drive/ME320Logs.csv')

In [ ]:
lastTimeStamp=pd.to_datetime(dataPDF['Time'].loc[0], dayfirst=True).to_pydatetime()

In [ ]:
lastDateStampStr=lastTimeStamp.strftime("%d-%m-%Y")
print(lastDateStampStr)

14-02-2022


# Analytics

In [ ]:
eventList=list(set(dataPDF['Event name'].to_list()))
eventList.sort()
componentList=list(set(dataPDF['Component'].to_list()))
componentList.sort()

In [ ]:
#Choose from dropdown menu eventList #Excluded login and 'A file has been uploaded.',
selectedTeachingEventsList=['Chapter created','Content created','Content uploaded','Course module created','Discussion created','Grade item created','Post created','Question created','Questions imported','Some content has been posted.','Wiki page created'] #,'Course module created','Some content has been posted.']
selectedCourseUpdateEventsList=['Course module updated','Course section created','Course section updated','Course section deleted']
selectedGradingEventsList=['Question manually graded','The submission has been graded.','User graded']
selectedLearningEventsList=['A file has been uploaded.','An online text has been uploaded.','A submission has been submitted.','Chapter created','Chapter viewed','Content created','Comments viewed','Content uploaded','Content viewed','Comment created','Course activity completion updated','Discussion created','Discussion viewed','Feedback viewed','H5P content viewed','Message sent','Post created','Quiz attempt submitted','mod_hvp: attempt submitted','Sessions viewed','Some content has been posted.','Submission created.','The submission has been graded.','User graded','Wiki page created','Wiki page viewed']#

## LMS Interaction

In [ ]:
usrType='student' #'faculty' #'student' #Pick from list ['faculty','student']

In [ ]:
gpList=list(set(usersPDF['profile_field_google_group'].to_list()))
gpList.sort()
if usrType=='student':
  fieldsList=selectedLearningEventsList
  usrGp='TE-2025A-B8' #'All' #'ETech-2023-B6'# Choose from drop down list gpList+['All']
  if usrGp!='All':
    trrmp=usersPDF[usersPDF['profile_field_google_group']==usrGp]['User full name'].to_list()
  else:
    trrmp=listStudents

  trrmp.sort()
  users=trrmp
  
else:
  fieldsList=selectedTeachingEventsList #+selectedCourseUpdateEventsList #Pick from [selectedTeachingEventsList,selectedGradingEventsList]
  users=listFaculty
  usrGp='academic'


#fldFilteredPDF=dataPDF[((dataPDF[fldName] in fieldsList) & (dataPDF['User full name'] in users))]

In [ ]:
dfc=[nn for nn in dataPDF.index.values if (dataPDF['User full name'].loc[nn] in users)]
fldFilteredPDF=dataPDF.loc[dfc]

In [ ]:
fig00, trry0=grouped_itemized_graphs(fldFilteredPDF, 'Component', componentList, 'Event name', fieldsList, 'Numbers', figTitle='Overview '+usrGp+' '+usrType+' LMS activity - as of '+lastDateStampStr+':2400')

In [ ]:
#fig00.write_html('/content/drive/My Drive/SLTC/LMS Usage/User Stats/'+lastDateStampStr+'/Overview of-'+usrGp+'-'+usrType+'-LMS-activities-'+lastDateStampStr+'.html')

In [ ]:
userActivityFig, userActivityPDF=grouped_itemized_graphs(fldFilteredPDF, 'Event name', fieldsList, 'User full name', users, 'Numbers', figTitle='Number of: '+usrGp+' '+usrType+' LMS activity - as of '+lastDateStampStr+':2400')
userActivityPDF['Total activity count']=userActivityPDF.loc[:,fieldsList].sum(axis=1).to_list()

In [ ]:
#userActivityFig.write_html('/content/drive/My Drive/SLTC/LMS Usage/User Stats/'+lastDateStampStr+'/Number of-'+usrGp+'-'+usrType+'-wise-LMS-activities-'+lastDateStampStr+'.html')

In [ ]:
tempxPDF=fldFilteredPDF.copy(deep=True)
columnsList=['User active','Group']+componentList
aaPDF=pd.DataFrame(np.zeros([len(users),len(columnsList)]),index=users, columns=columnsList)
tmpGpList=[]
usrModuleViewTotal=[]
aaay=[]
for usr in users:
  if usr in list(set(tempxPDF['User full name'].to_list())):
    aaPDF.loc[usr,'User active']=1
    usrwisePDF=tempxPDF[tempxPDF['User full name']==usr]
    cn=usrwisePDF['Time'].to_list()
    cp=[cn[0]]+cn[:-1]
    trr=pd.to_datetime(cp, dayfirst=True).to_pydatetime()-pd.to_datetime(cn, dayfirst=True).to_pydatetime()
    tru=trr.tolist()
    usrwisePDF['usrdt']=[float("{:.2f}".format(((dt.seconds/60.))/60.)) for dt in tru]
    lbls=usrwisePDF['Component'].to_list()
    for lbl in componentList:
      aaPDF.loc[usr,lbl]=sum(usrwisePDF[usrwisePDF['Component']==lbl]['usrdt'])

  else:
    for lbls in columnsList:
      aaPDF.loc[usr,lbls]=0.
  
  tmpg0=usersPDF[usersPDF['User full name']==usr]['profile_field_google_group'].to_list()[0]
  aaPDF.loc[usr,'Group']=tmpg0
  
aaPDF['Total time / Hrs']=aaPDF.loc[:,componentList].sum(axis=1).to_list()
LMSuserStatsPDF=pd.concat([aaPDF,userActivityPDF],axis=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Download
#LMSuserStatsPDF.to_csv('/content/drive/My Drive/SLTC/LMS Usage/User Stats/'+lastDateStampStr+'/Number of LMS activity and time spent: '+usrGp+'-'+usrType+'-'+lastDateStampStr+'.csv')

In [ ]:
LMSuserStatsPDF

## Given the student group Course wise LMS interaction

In [ ]:
idxs=[]
for nn in fldFilteredPDF.index.values:
  if (fldFilteredPDF['Event context'].loc[nn].split(':')[0]=='Course'):
    idxs+=[nn]
courseFldFilteredPDF=fldFilteredPDF.loc[idxs]
csrs=list(set(courseFldFilteredPDF['Event context'].loc[idxs].to_list()))
csrs.sort()

In [ ]:
fig1, trrz=grouped_itemized_graphs(courseFldFilteredPDF, 'Event name', fieldsList, 'Event context', csrs, 'Numbers', figTitle='Number of course wise: '+usrGp+' '+usrType+' LMS activity - as of '+lastDateStampStr+':2400')

In [ ]:
#fig1.write_html('/content/drive/My Drive/SLTC/LMS Usage/User Stats/'+lastDateStampStr+'/Number of course wise-'+usrGp+'-'+usrType+'-LMS-activities-'+lastDateStampStr+'.html')

In [ ]:
#trrz.to_csv('/content/drive/My Drive/SLTC/LMS Usage/User Stats/'+lastDateStampStr+'/Number of course wise-'+usrGp+'-'+usrType+'-LMS-activities-'+lastDateStampStr+'.csv')

# Given a course find stats

In [ ]:
dfs=[nn for nn in dataPDF.index.values if (dataPDF['User full name'].loc[nn] in listStudents)]
studentFilteredPDF=dataPDF.loc[dfs]

In [ ]:
idxs=[]
for nn in studentFilteredPDF.index.values:
  if (studentFilteredPDF['Event context'].loc[nn].split(':')[0]=='Course'):
    idxs+=[nn]
allCourseFldFilteredPDF=studentFilteredPDF.loc[idxs]
allCsrs=list(set(studentFilteredPDF['Event context'].loc[idxs].to_list()))
allCsrs.sort()

In [ ]:
# Pick course from drop down list allCsrs
crs='Course: Industry Exposure - Class of 2023' #'Course: [2025A:S3]-SMA2204-Advanced Calculus-Multivariable integration (2C)'
crsFilteredPDF=allCourseFldFilteredPDF[allCourseFldFilteredPDF['Event context']==crs]
crsUsers=list(set(crsFilteredPDF['User full name'].to_list()))
crsUsers.sort()

In [ ]:
len(trrw)

39

In [ ]:
fig12, trrw=grouped_itemized_graphs(crsFilteredPDF, 'Event name', fieldsList, 'User full name', crsUsers, 'Numbers', figTitle='LMS activity -'+crs+'- as of '+lastDateStampStr+':2400')

In [ ]:
trrw

In [ ]:
trrw.to_csv('/content/drive/My Drive/test.csv')

In [ ]:
trrw.sum(axis=1).sum()

0

#Temp stuff

In [ ]:
xx1=users
traceNames1=fieldsList
yy1=[]
for lbls in traceNames1:
  yy1+=[LMSuserStatsPDF[lbls].to_list()]

fig03 = go.Figure()
for i,trNm in enumerate(traceNames1):
  fig03.add_trace(go.Bar(y=yy1[i], x=xx1, name=trNm))

figTitle='LMS activity count: '+usrGp+' '+usrType+' - as of '+lastDateStampStr+':2400' #Input field
yLable='Number'
fig03.update_layout(title=figTitle, xaxis_title='', yaxis_title=yLable, barmode='stack')
fig03.show()
#fig=bar_plot(xx1,yy1,traceNames1,figTitle,'',yLabel,'stack')

In [ ]:
xx2=users
traceNames2=componentList
yy2=[]
for lbls in traceNames2:
  yy2+=[LMSuserStatsPDF[lbls].to_list()]

fig04 = go.Figure()
for i,trNm in enumerate(traceNames2):
  fig04.add_trace(go.Bar(y=yy2[i], x=xx2, name=trNm))

figTitle='Time spent: '+usrGp+' '+usrType+' - as of '+lastDateStampStr+':2400' #Input field
yLable='Hours'
fig04.update_layout(title=figTitle, xaxis_title='', yaxis_title=yLable, barmode='stack')
fig04.show()
#fig=bar_plot(xx1,yy1,traceNames1,figTitle,'',yLabel,'stack')

In [ ]:
xx3=csrs
traceNames3=columnsList
yy3=[]
for lbls in traceNames3:
  yy3+=[LMCourseStatsPDF[lbls].to_list()]

fig05 = go.Figure()
for i,trNm in enumerate(traceNames3):
  fig05.add_trace(go.Bar(y=yy3[i], x=xx3, name=trNm))

figTitle='Number of activities: course wise - '+usrGp+usrType+' - as of '+lastDateStampStr+':2400' #Input field
yLable='Numbers'
fig05.update_layout(title=figTitle, xaxis_title='', yaxis_title=yLable, barmode='stack')
fig05.show()
#fig=bar_plot(xx1,yy1,traceNames1,figTitle,'',yLabel,'stack')

# Learning analytics

In [ ]:
from plotly.subplots import make_subplots
from numpy import linalg as lin
import sklearn
from sklearn.mixture import GaussianMixture

##RL Functions

In [ ]:
class mugas_data_analysis:
  def __init__(self):
    self=[];
  
  def grouped_bar_plot(self, dataPDF, gpFieldName, plotFieldName, figTitle):
    xx1=[]; yy1=[]; cc1=[]; traceNames1=[];
    gpFldList=list(set(dataPDF[gpFieldName].tolist()))
    gpFldList.sort()
    for xx in gpFldList:
      temp=dataPDF[dataPDF[gpFieldName]==xx];
      xxx1=[]; yyy1=[]; ccc1=[];
      if plotFieldName!='':
        plotFldList=list(set(temp[plotFieldName].to_list()))
        plotFldList.sort()
        for tstdt in plotFldList:
          xxx1+=[tstdt]
          ccc1+=[temp[temp[plotFieldName]==tstdt]];
          yyy1+=[len(temp[temp[plotFieldName]==tstdt])];
      else:
        xxx1=[gpFieldName]
        ccc1+=[temp[temp[gpFieldName]==xx]];
        yyy1+=[len(temp[temp[gpFieldName]==xx])];
      xx1+=[xxx1];
      yy1+=[yyy1];
      cc1+=[ccc1]
      traceNames1+=[xx]
      

    fig=self.bar_plot(xx1,yy1,traceNames1,plotFieldName+' '+figTitle,'','Numbers','stack')
    return fig

  def hist_plot(self, xx, binSize, binRange, traceNames, titleText, xLabel,yLabel):
      fig = go.Figure()
      
      for i,x in enumerate(xx):
        fig.add_trace(go.Histogram(
            x=x,
            #histnorm='percent',
            #cumulative_enabled=cumulative,
            name=traceNames[i], # name used in legend and hover labels
            xbins=dict( # bins used for histogram
                start=binRange[0],
                end=binRange[1],
                size=binSize
            ),
            #marker_color='#EB89B5',
            opacity=0.75
        ))

      fig.update_layout(
          title_text=titleText, # title of plot
          xaxis_title_text=xLabel, # xaxis label
          yaxis_title_text=yLabel, # yaxis label
          bargap=0.2, # gap between bars of adjacent location coordinates
          bargroupgap=0.1 # gap between bars of the same location coordinates
      )

      fig.show()
      #fig.write_html("/content/drive/My Drive/LMS Logs/Course_STATS_upto_May/"+titleText+".html")
      return fig

  def scatter_plot(self, xx, yy, mode, traceName, figTitle, xLabel, yLabel):
    fig = go.Figure()
    for i,x in enumerate(xx):
      fig.add_trace(go.Scatter(y=yy[i], x=x, mode=mode, name=traceName[i] ))
    
    
    fig.update_layout(
        title=figTitle, xaxis_title=xLabel, yaxis_title=yLabel,
        xaxis=dict(
            dtick=.01,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
        ),
        yaxis=dict(
            dtick=1,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
            visible=False
        )
    )
    fig.show()
    return fig

  def scatter_plot3D(self,xx,yy,zz,color):
    fig = go.Figure(data=[go.Scatter3d(x=xx, y=yy, z=zz, 
                                    mode='markers',marker=dict(
          #size=12,
          color=color,                # set color to an array/list of desired values
          #colorscale='Viridis',   # choose a colorscale
          opacity=0.5
      ))])
    fig.show()
    return fig

  def bar_plot(self, xx, yy, traceName, figTitle, xLabel, yLabel, barmode):
    fig = go.Figure()
    for i,x in enumerate(xx):
      fig.add_trace(go.Bar(y=yy[i], x=x, name=traceName[i]))
    
    
    fig.update_layout(
        title=figTitle, xaxis_title=xLabel, yaxis_title=yLabel, barmode=barmode
    )
    fig.show()
    return fig


  def principle_component_values(self, modelDF, predictDF, nnPCs):
    tempModelDataPDF=modelDF.copy();
    tempPredictDataPDF=predictDF.copy();
    #nnPCs=min(len(tempModelDataPDF.index.values),len(tempModelDataPDF.columns));
    #nnPCs='mle'
    pca=sklearn.decomposition.PCA(n_components=nnPCs);
    modelnumPCA=pca.fit_transform(tempModelDataPDF);
    PCweights=pca.singular_values_;
    PCAmodelScore=pca.score(tempModelDataPDF);
    predictnumPCA=pca.transform(tempPredictDataPDF);
    PCApredictScore=pca.score(tempPredictDataPDF);
    #scaling=sklearn.preprocessing.StandardScaler().fit(pd.DataFrame(numPCA));
    #scaledPCA=scaling.transform(numPCA);
    transformer = sklearn.decomposition.FactorAnalysis(n_components=nnPCs); # n_components=nnPCs, random_state=0);
    modelsvdFactor = transformer.fit_transform(tempModelDataPDF);
    svdFactormodelScore=transformer.score(tempModelDataPDF);
    predictsvdFactor = transformer.transform(tempPredictDataPDF);
    svdFactorpredictScore=transformer.score(tempPredictDataPDF);
    
    
    mindex=tempModelDataPDF.index.values;
    pindex=tempPredictDataPDF.index.values;
    #figFactor=self.hist_plot(PPFactor.T,histBinSize, binRange, traceNames,'Distributions of the PC Factors','PC Factors','Frequency')
    pcaOutput={'fitScores': {'PCApredictScore':PCApredictScore, 'PCAmodelScore':PCAmodelScore, 'svdFactormodelScore':svdFactormodelScore, 'svdFactorpredictScore':svdFactorpredictScore}, 'PCweights':PCweights, 'modelnumPCA':pd.DataFrame(modelnumPCA,index=mindex),'predictnumPCA':pd.DataFrame(predictnumPCA,index=pindex), 'modelsvdFactor':pd.DataFrame(modelsvdFactor,index=mindex),  'predictsvdFactor':pd.DataFrame(predictsvdFactor,index=pindex)}
    return pcaOutput

  def gmm_data_model(self, mData, nClusters):
    #mData - numeric centered np array of shape (ndata,nfactors)
    
    gmm = sklearn.mixture.GaussianMixture(n_components = nClusters)
    gmm.fit(mData)
    labels = gmm.predict(mData)
    clusterIC=[gmm.aic(mData), gmm.bic(mData)]
    print(clusterIC)
    fig1=md.scatter_plot3D(mData[:,0],mData[:,1],mData[:,2],labels)
    
    return {'gmm':gmm, 'clusterIC':clusterIC}


  def gmm_data_classifier(self, model_DataPDF, predict_DataPDF,nClusters):
    modelDataPDF0=model_DataPDF.copy();
    predictDataPDF0=predict_DataPDF.copy();
    modelData=modelDataPDF0.to_numpy();
    predictData=predictDataPDF0.to_numpy();
    nPCs=modelData.shape[1]
    #print([modelPCA[n].shape,predictPCA[n].shape])
    fitGMM=self.gmm_data_model(modelData,nClusters)
    gmm=fitGMM['gmm'];
    
    #Plabels=[];
    #for nn in range(nPCs):
    #    Plabels+=['PF'+str(nn)]
    
    mLabels = gmm.predict(modelData)
    #modelFitScore=gmm.score(modelData);
    modelDist={'Model Mean':gmm.means_, 'Model Covaraince':gmm.covariances_};
    modelSampleFitScore=gmm.score_samples(modelData);
    modelDataPDF0['labels']=mLabels
    modelDataPDF0['Fit Score']=modelSampleFitScore;

    pLabels = gmm.predict(predictData)
    #predictFitScore=gmm.score(predictData);
    predictProbDist=gmm.predict_proba(predictData);
    predictSampleFitScore=gmm.score_samples(predictData);
    
    predictDataPDF0['labels']=pLabels
    predictDataPDF0['Fit Score']=predictSampleFitScore;
    xx1=[]; yy1=[]; traceNames1=[];
    xx2=[]; yy2=[]; traceNames2=[];

    for kk in set(modelDataPDF0['labels'].to_list()):
      xx1+=[['All Data']];
      yy1+=[[len(modelDataPDF0[modelDataPDF0['labels']==kk])]];
      yy2+=[[len(predictDataPDF0[predictDataPDF0['labels']==kk])]];
      traceNames1+=['GP'+str(kk)]
    fig21=self.bar_plot(xx1,yy1,traceNames1,'Model Clusters','','Numbers','stack')
    fig31=self.bar_plot(xx1,yy2,traceNames1,'Predicted Clusters','','Numbers','stack')
    #print({'modelFitScore':modelFitScore, 'predictFitScore':predictFitScore})
    return {'labeled model data':modelDataPDF0, 'labeled predict data':predictDataPDF0,'fitScore':{'modelFitScore':modelSampleFitScore, 'predictFitScore':predictSampleFitScore},'Probability Model':modelDist}

  def logistic_reg_data_classifier(self, model_XDataPDF, model_YDataPDF, predict_XDataPDF):
    MXDataPDF=model_XDataPDF.copy();
    MYDataPDF=model_YDataPDF.copy();
    PXDataPDF=predict_XDataPDF.copy();
    clf = sklearn.linear_model.LogisticRegression(random_state=0).fit(MXDataPDF.to_numpy(), MYDataPDF.to_numpy());
    mLabels = clf.predict(MXDataPDF.to_numpy())
    modelDist=clf.predict_proba(MXDataPDF.to_numpy());
    modelSampleFitScore=clf.score(MXDataPDF.to_numpy(), MYDataPDF.to_numpy());
    MXDataPDF['labels']=mLabels
    mLabelTypes=list(set(mLabels));
    probLabels=[];
    for mlt in mLabelTypes:
      probLabels+=['p('+str(mlt)+')']
    MXDataPDF[probLabels]=modelDist;

    pLabels = clf.predict(PXDataPDF.to_numpy())
    predictDist=clf.predict_proba(PXDataPDF.to_numpy());
    pLabelTypes=set(probLabels);
    PXDataPDF['labels']=pLabels;
    pLabelTypes=list(set(pLabels));
    probLabels=[];
    for mlt in pLabelTypes:
      probLabels+=['p('+str(mlt)+')']

    PXDataPDF[probLabels]=predictDist;
    
    xx1=[]; yy1=[]; traceNames1=[];
    xx2=[]; yy2=[]; traceNames2=[];

    for kk in set(MXDataPDF['labels'].to_list()):
      xx1+=[['All Data']];
      yy1+=[[len(MXDataPDF[MXDataPDF['labels']==kk])]];
      yy2+=[[len(PXDataPDF[PXDataPDF['labels']==kk])]];
      traceNames1+=['GP-'+str(kk)]
    fig21=self.bar_plot(xx1,yy1,traceNames1,'Model Clusters','','Numbers','stack')
    fig31=self.bar_plot(xx1,yy2,traceNames1,'Predicted Clusters','','Numbers','stack')
    
    
    #print({'modelFitScore':modelFitScore, 'predictFitScore':predictFitScore})
    return {'labeled model data':MXDataPDF, 'labeled predict data':PXDataPDF,'Probability Model':{'modelDist':modelDist,'predictDist':predictDist}}

##RL Analysis

In [ ]:
md=mugas_data_analysis()

In [ ]:
RLdataPDF=pd.DataFrame(LMSuserStatsPDF[fieldsList+componentList].to_numpy(),index=LMSuserStatsPDF.index.values)

In [ ]:
yyr=md.principle_component_values(RLdataPDF,RLdataPDF,10)  

In [ ]:
[*yyr]

['traceNames',
 'fitScores',
 'PCweights',
 'modelnumPCA',
 'predictnumPCA',
 'modelsvdFactor',
 'predictsvdFactor']

In [ ]:
tempaPDF=yyr['predictsvdFactor']
xx=[]
traceNames=[]
for zz in tempaPDF.columns:
  xx+=[tempaPDF[zz]]
  traceNames+=['PCA#'+str(zz)]


In [ ]:
figaa=md.hist_plot(xx,5,[-150,150], traceNames,'test','xlabel','ylabel')

In [ ]:
clstrs=md.gmm_data_classifier(tempaPDF,tempaPDF,2)

[1775.2488964037661, 2073.4921579948696]


In [ ]:
[*clstrs]

['labeled model data', 'labeled predict data', 'fitScore', 'Probability Model']

In [ ]:
clstrs['labeled model data']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,labels,Fit Score
21UG0004 JANANI NUPEARACHCHI,-37.965508,-39.754490,-0.148320,9.745567,-7.638437,0.871939,-0.453833,2.170935,-2.996946,2.644984,-0.597383,-0.834629,-0.039421,0.297393,-0.243538,0,-19.497323
21UG0012 NIMESH DILSHAN,-23.011957,-20.497064,21.165306,29.382111,-0.437501,5.289870,-2.106308,-10.768501,-7.136091,-2.161440,-1.442391,0.008532,0.319104,-0.378813,-0.112389,0,-35.895754
21UG0019 TISHAN KALUWALGODA,-17.373324,-3.092158,0.826896,15.931219,-10.164930,3.970951,-3.171978,3.597207,-4.849692,4.529513,-1.393160,-1.308441,-0.183889,0.410822,-0.641057,0,-24.995767
21UG0020 SIDATH DIYANATHA,-44.687962,-50.689728,-4.273267,-0.673570,-0.255503,-1.741208,-0.837825,-0.064060,-2.053761,-0.195331,0.099936,-0.460447,0.002854,-0.057745,0.087312,0,-17.966427
21UG0022 VIBHUTHI EKANAYAKA,-2.882314,43.012234,-20.359002,-2.497693,1.342562,8.262436,2.352112,0.494387,2.549108,-3.545270,1.082988,9.666546,0.533313,-0.827525,-0.184831,3,-43.205932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21UG0834 M.G.C.S.C. BANDARA,-22.065561,-11.414819,-0.792170,-6.570660,-0.525865,-4.210471,-4.956379,-1.162763,8.145998,0.939047,-3.991203,-0.003638,0.050223,-0.938599,0.044322,0,-34.111575
21UG0835 H.G.C.L. HARISCHANDRA,18.789554,55.241871,5.783764,-15.762833,-2.198610,2.259818,-8.492025,-0.993289,-2.794400,-1.561146,-0.778195,-0.102074,-0.120791,-0.365628,-0.241698,3,-36.631223
21UG0868 R.D.R.S. RAJAPAKSHA,2.795604,10.991641,1.871912,-5.515554,9.567991,-5.178257,-1.143606,11.218701,-4.567667,-3.209253,0.452586,-0.101843,0.317564,-0.803262,0.036712,3,-35.288663
21UG0869 M.G.U. SEWWANDI,168.796091,-16.323379,32.400131,37.841873,-0.070493,-6.657092,2.229598,-17.316078,1.668654,-4.318580,-5.016458,1.977012,0.583409,-2.053709,-0.184022,4,85.555585
